In [0]:
!pip install geopandas

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 315kB/s 
     |████████████████████████████████| 10.4MB 41.3MB/s 


In [0]:
import pandas as pd
import geopandas as gpd
from google.colab import files

#  Gets the current uploaded counts
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  todays_counts = pd.read_csv(fn)


todays_counts['ID'] = todays_counts['ID'].str.upper()
todays_counts.fillna(0, inplace=True)
todays_counts.head()

Saving data-6qBRk (6).csv to data-6qBRk (6).csv
User uploaded file "data-6qBRk (6).csv" with length 1254 bytes


,ID,Value
0,RICHMOND,0.0
1,ALEXANDER,0.0
2,DURHAM,1.0
3,VANCE,0.0
4,ROCKINGHAM,0.0


In [0]:
display(todays_counts[todays_counts['Value'] > 0])

,ID,Value
2,DURHAM,1.0
6,WILSON,1.0
22,HARNETT,3.0
24,ONSLOW,1.0
31,SAMPSON,1.0
33,WAYNE,1.0
42,FORSYTH,2.0
61,MECKLENBURG,7.0
67,JOHNSTON,2.0
72,BRUNSWICK,1.0


In [0]:
# Get geometry of counties
nc_counties = gpd.read_file("https://opendata.arcgis.com/datasets/34acbf4a26784f189c9528c1cf317193_0.geojson")

# Get dataset with historical values
nc_timeline = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQn6xaVpOcC0TpFaH_BkQy0Av1ZJDeFfziD6iYR6sC-kiMSMz65J6uo9rh8dIS_sTQdh1RPFp9QMDBo/pub?gid=0&single=true&output=csv')
nc_timeline['ID'] = nc_timeline['ID'].str.upper()

# Combine datasets
df = nc_counties.merge(todays_counts, left_on = 'CO_NAME', right_on = 'ID')
df = df[['CO_NAME', 'ID', 'geometry', 'Value']]

# timeline = df.merge(nc_timeline, left_on = 'ID', right_on = 'ID')
# timeline.head()
# filename = 'timeline.csv'
# from google.colab import files
# timeline.to_csv(filename)
# files.download(filename)

In [0]:
import folium
import branca.colormap as cm
from datetime import date

m = folium.Map(
    location=[df.geometry.centroid.y.mean(), df.geometry.centroid.x.mean()], zoom_start=7, tiles=None)

folium.TileLayer(tiles='openstreetmap', show=True, control=False, min_zoom=5).add_to(m)

#https://nbviewer.jupyter.org/gist/jtbaker/c9b94010a160bd2120f5b3dde082ddf1

today = date.today().strftime("%m/%d/%Y")
search_value = 'Value'
search_label = 'Confirmed Cases (' + str(today) +")"

# print(colormap(5.0))

colormap = cm.LinearColormap(colors=['white','pink','red'], index=[df[search_value].min(),df[search_value].min()+1,df[search_value].max()],vmin=df[search_value].min(),vmax=df[search_value].max())
    
colormap

fg = folium.FeatureGroup(name=search_label, show=True)

style_function=lambda x: {
  'fillColor': colormap(x['properties']['Value']),
  'color': 'black',
  'weight': 1,
  'fillOpacity': 0.45,
  'weight': 1,
  'opacity':0.4,
  'nan_fill_color': 'purple'
  }

folium.GeoJson(df, 
    style_function=style_function,

    tooltip=folium.GeoJsonTooltip(fields=['ID','Value'],
                                            aliases=['<div style="background-color: white; color: black; padding: 3px; border: 0px solid black; border-radius: 5px;">'+item+'</div>' for item in ['County','Confirmed Cases']],
                                            style="font-family: Helvetica, Arial, sans-serif;",
                                          localize=True),
              highlight_function=lambda x: {"weight":3,'fillOpacity':0.8}
    ).add_to(fg)

fg.add_to(m)
colormap.caption = search_label
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

m

filename = date.today().strftime("%Y%m%d")+ '_NC_COVID.html'
from google.colab import files
m.save(filename)
files.download(filename)